# Import Libraries and Data

In [79]:
#!pip install geopandas

In [80]:
import pandas as pd
import geopandas as gpd # type: ignore
import plotly.express as px
import plotly.graph_objects as go

In [81]:
# Read in the data
df1 = pd.read_csv('Wildfire Data.csv', dtype={'Battalion': str, 'Fire Name (Secondary)': str, 'APN (parcel)': str})
df2 = gpd.read_file('Postfire_Master_Data_Share.geojson')

# Data Joining and Preprocessing

Dataset Link: https://www.kaggle.com/datasets/vijayveersingh/the-california-wildfire-data

In [82]:
# Preprocess Columns in DF1

# Strip * and then leading spaces from start of column names
df1.columns = df1.columns.str.lstrip('*').str.lstrip()

# Drop ID columns since Pandas will add its own
df1.drop(columns=['_id','OBJECTID'], inplace=True)

# Remove parentheses (and text inside) from column names
df1.columns = df1.columns.str.replace(r"\(.*\)", "", regex=True).str.strip()

# Rename certain columns
df1.rename(columns={'If Affected 1-9% - Where did fire start?':'Fire Start Location',
                    'If Affected 1-9% - What started fire?' : 'Fire Cause'}, inplace=True)

df1.columns.tolist(), df1.shape

(['Damage',
  'Street Number',
  'Street Name',
  'Street Type',
  'Street Suffix',
  'City',
  'State',
  'Zip Code',
  'CAL FIRE Unit',
  'County',
  'Community',
  'Battalion',
  'Incident Name',
  'Incident Number',
  'Incident Start Date',
  'Hazard Type',
  'Fire Start Location',
  'Fire Cause',
  'Structure Defense Actions Taken',
  'Structure Type',
  'Structure Category',
  '# Units in Structure',
  '# of Damaged Outbuildings < 120 SQFT',
  '# of Non Damaged Outbuildings < 120 SQFT',
  'Roof Construction',
  'Eaves',
  'Vent Screen',
  'Exterior Siding',
  'Window Pane',
  'Deck/Porch On Grade',
  'Deck/Porch Elevated',
  'Patio Cover/Carport Attached to Structure',
  'Fence Attached to Structure',
  'Distance - Propane Tank to Structure',
  'Distance - Residence to Utility/Misc Structure &gt; 120 SQFT',
  'Fire Name',
  'APN',
  'Assessed Improved Value',
  'Year Built',
  'Site Address',
  'GLOBALID',
  'Latitude',
  'Longitude',
  'x',
  'y'],
 (100230, 45))

In [83]:
# Preprocess Columns in DF2

df2.drop(columns=['OBJECTID'], inplace=True)

rename_mapping = {
    "DAMAGE": "Damage",
    "STREETNUMBER": "Street Number",
    "STREETNAME": "Street Name",
    "STREETTYPE": "Street Type",
    "STREETSUFFIX": "Street Suffix",
    "CITY": "City",
    "STATE": "State",
    "ZIPCODE": "Zip Code",
    "CALFIREUNIT": "Calfire Unit",
    "COUNTY": "County",
    "COMMUNITY": "Community",
    "BATTALION": "Battalion",
    "INCIDENTNAME": "Incident Name",
    "INCIDENTNUM": "Incident Num",
    "INCIDENTSTARTDATE": "Incident Start Date",
    "HAZARDTYPE": "Hazard Type",
    "WHEREFIRESTARTEDONSTRUCTURE": "Where Fire Started On Structure",
    "WHATDIDFIRESTARTFROM": "What Did Fire Start From",
    "DEFENSIVEACTIONS": "Defensive Actions",
    "STRUCTURETYPE": "Structure Type",
    "STRUCTURECATEGORY": "Structure Category",
    "NUMBEROFUNITPERSTRUCTURE": "Number Of Unit Per Structure",
    "NOOUTBUILDINGSDAMAGED": "No Out Buildings Damaged",
    "NOOUTBUILDINGSNOTDAMAGED": "No Out Buildings Not Damaged",
    "ROOFCONSTRUCTION": "Roof Construction",
    "EAVES": "Eaves",
    "VENTSCREEN": "Ventscreen",
    "EXTERIORSIDING": "Exterior Siding",
    "WINDOWPANE": "Window Pane",
    "DECKPORCHONGRADE": "Deck Porch On Grade",
    "DECKPORCHELEVATED": "Deck Porch Elevated",
    "PATIOCOVERCARPORT": "Patio Cover Carport",
    "FENCEATTACHEDTOSTRUCTURE": "Fence Attached To Structure",
    "PROPANETANKDISTANCE": "Propane Tank Distance",
    "UTILITYMISCSTRUCTUREDISTANCE": "Utility Misc Structure Distance",
    "FIRENAME": "Fire Name",
    "APN": "Apn",
    "ASSESSEDIMPROVEDVALUE": "Assessed Improved Value",
    "YEARBUILT": "Year Built",
    "SITEADDRESS": "Site Address",
    "GLOBALID": "GLOBALID",  
    "LATITUDE": "Latitude",
    "LONGITUDE": "Longitude",
    "GEOMETRY": "Geometry"
}

df2.rename(columns=rename_mapping, inplace=True)
df2.columns.tolist(), df2.shape

(['Damage',
  'Street Number',
  'Street Name',
  'Street Type',
  'Street Suffix',
  'City',
  'State',
  'Zip Code',
  'Calfire Unit',
  'County',
  'Community',
  'Battalion',
  'Incident Name',
  'Incident Num',
  'Incident Start Date',
  'Hazard Type',
  'Where Fire Started On Structure',
  'What Did Fire Start From',
  'Defensive Actions',
  'Structure Type',
  'Structure Category',
  'Number Of Unit Per Structure',
  'No Out Buildings Damaged',
  'No Out Buildings Not Damaged',
  'Roof Construction',
  'Eaves',
  'Ventscreen',
  'Exterior Siding',
  'Window Pane',
  'Deck Porch On Grade',
  'Deck Porch Elevated',
  'Patio Cover Carport',
  'Fence Attached To Structure',
  'Propane Tank Distance',
  'Utility Misc Structure Distance',
  'Fire Name',
  'Apn',
  'Assessed Improved Value',
  'Year Built',
  'Site Address',
  'GLOBALID',
  'Latitude',
  'Longitude',
  'geometry'],
 (100230, 44))

In [84]:
# Merge Datasets 

merged_df = df1.merge(df2, on="GLOBALID", how="inner", suffixes=('', '_df2'))

# First, for columns with identical names (except the join key and geometry),
# fill in missing values in df1 using the corresponding df2 column.
overlap_cols = set(df1.columns).intersection(df2.columns) - {"GLOBALID", "geometry"}
for col in overlap_cols:
    col_df2 = col + '_df2'
    if col_df2 in merged_df.columns:
        merged_df[col] = merged_df[col].fillna(merged_df[col_df2])
        # Drop the duplicate column after imputation
        merged_df.drop(columns=[col_df2], inplace=True)

# Next, handle columns that have similar but not identical names.
# Create a mapping where the key is the df1 column to fill and the value is the df2 column.
manual_mapping = {
    'Vent Screen': 'Ventscreen',
    'Deck/Porch On Grade': 'Deck Porch On Grade',
    'Deck/Porch Elevated': 'Deck Porch Elevated',
    'Patio Cover/Carport Attached to Structure': 'Patio Cover Carport',
    'Distance - Propane Tank to Structure': 'Propane Tank Distance',
    'Distance - Residence to Utility/Misc Structure &gt; 120 SQFT': 'Utility Misc Structure Distance',
    'APN': 'Apn'
}

for col_df1, col_df2 in manual_mapping.items():
    if col_df2 in merged_df.columns:
        merged_df[col_df1] = merged_df[col_df1].fillna(merged_df[col_df2])
        # Drop the df2 column after filling
        merged_df.drop(columns=[col_df2], inplace=True)

# Optionally, if there remain any duplicate columns that weren't handled by the fillna,
# list them explicitly and drop them.
columns_to_drop = [
    'Ventscreen', 
    'Deck Porch On Grade', 
    'Deck Porch Elevated', 
    'Patio Cover Carport', 
    'Propane Tank Distance', 
    'Utility Misc Structure Distance', 
    'Apn'
]
merged_df = merged_df.drop(columns=columns_to_drop, errors='ignore')

# Display the remaining columns and shape
merged_df.columns.tolist(), merged_df.shape

(['Damage',
  'Street Number',
  'Street Name',
  'Street Type',
  'Street Suffix',
  'City',
  'State',
  'Zip Code',
  'CAL FIRE Unit',
  'County',
  'Community',
  'Battalion',
  'Incident Name',
  'Incident Number',
  'Incident Start Date',
  'Hazard Type',
  'Fire Start Location',
  'Fire Cause',
  'Structure Defense Actions Taken',
  'Structure Type',
  'Structure Category',
  '# Units in Structure',
  '# of Damaged Outbuildings < 120 SQFT',
  '# of Non Damaged Outbuildings < 120 SQFT',
  'Roof Construction',
  'Eaves',
  'Vent Screen',
  'Exterior Siding',
  'Window Pane',
  'Deck/Porch On Grade',
  'Deck/Porch Elevated',
  'Patio Cover/Carport Attached to Structure',
  'Fence Attached to Structure',
  'Distance - Propane Tank to Structure',
  'Distance - Residence to Utility/Misc Structure &gt; 120 SQFT',
  'Fire Name',
  'APN',
  'Assessed Improved Value',
  'Year Built',
  'Site Address',
  'GLOBALID',
  'Latitude',
  'Longitude',
  'x',
  'y',
  'Calfire Unit',
  'Incident N

In [85]:
# Show the columns with null values in descending order
null_columns = merged_df.isnull().sum().sort_values(ascending=False)
null_columns[null_columns > 0]

Battalion                                                       93551
Fire Cause                                                      91214
What Did Fire Start From                                        90904
Fire Start Location                                             89490
Where Fire Started On Structure                                 89177
Fire Name                                                       76883
Structure Defense Actions Taken                                 75760
Defensive Actions                                               74826
Distance - Residence to Utility/Misc Structure &gt; 120 SQFT    74040
No Out Buildings Not Damaged                                    69157
# of Non Damaged Outbuildings < 120 SQFT                        69157
No Out Buildings Damaged                                        69145
# of Damaged Outbuildings < 120 SQFT                            69145
# Units in Structure                                            69046
Number Of Unit Per S

In [86]:
# Drop rows missing key columns 
columns_to_drop = [
    'Vent Screen', 'Eaves', 'Window Pane', 'Exterior Siding', 
    'Roof Construction', 'APN', 'County', 'State'
]
merged_df = merged_df.dropna(subset=columns_to_drop)

# Drop Street Number column since it is not useful for analysis
merged_df = merged_df.drop(columns=['Street Number'], errors='ignore')

# Fill high-missing categorical columns with 'Unknown'
categorical_columns = [
    'Battalion', 
    'Fire Cause', 
    'What Did Fire Start From',
    'Fire Start Location', 
    'Where Fire Started On Structure', 
    'Fire Name',
    'Structure Defense Actions Taken', 
    'Defensive Actions',
    'Street Suffix', 
    'Zip Code', 
    'Community', 
    'Street Type', 
    'Street Name', 
    'City', 
    'Site Address'
]
for col in categorical_columns:
    if col in merged_df.columns:
        merged_df[col] = merged_df[col].fillna('Unknown')

# For high-missing numerical count-like columns, convert to numeric and fill with 0 
count_columns = [
    'No Out Buildings Not Damaged', 
    '# of Non Damaged Outbuildings < 120 SQFT',
    'No Out Buildings Damaged', 
    '# of Damaged Outbuildings < 120 SQFT',
    '# Units in Structure', 
    'Number Of Unit Per Structure'
]
for col in count_columns:
    if col in merged_df.columns:
        merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
        merged_df[col] = merged_df[col].fillna(0)

# For distance columns, convert to numeric and fill missing values with the median
distance_columns = [
    'Distance - Residence to Utility/Misc Structure &gt; 120 SQFT',
    'Distance - Propane Tank to Structure'
]
for col in distance_columns:
    if col in merged_df.columns:
        merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
        median_val = merged_df[col].median()
        if pd.isna(median_val):  # fallback if median is NaN
            median_val = 0
        merged_df[col] = merged_df[col].fillna(median_val)

# Convert additional numeric columns and fill missing values with the median
numeric_cols = [
    'Assessed Improved Value', 
    'Year Built'
]
for col in numeric_cols:
    if col in merged_df.columns:
        merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
        median_val = merged_df[col].median()
        merged_df[col] = merged_df[col].fillna(median_val)

# Imputation for remaining categorical columns
if 'Fence Attached to Structure' in merged_df.columns:
    merged_df['Fence Attached to Structure'] = merged_df['Fence Attached to Structure'].fillna('None')

merged_df.isnull().sum()

Damage                                                          0
Street Name                                                     0
Street Type                                                     0
Street Suffix                                                   0
City                                                            0
State                                                           0
Zip Code                                                        0
CAL FIRE Unit                                                   0
County                                                          0
Community                                                       0
Battalion                                                       0
Incident Name                                                   0
Incident Number                                                 0
Incident Start Date                                             0
Hazard Type                                                     0
Fire Start

# Proposed Responsibilities and Timeline

**Ethan:** Write the project proposal after discussing as a group what the research question/problem statement/suggestion is along with the intended audience. Also, once project is nearly complete, will write the summary of data cleaning and transformation steps, key insights from exploratory analysis and justification of any data exclusions or assumptions (if any). If new dataset is picked, process it if needed for Timmy and Andy to use. 

**Timmy and Andy:** Work on the primary visualizations in Python to effectively communicate insights. For each graph, just bullet point reasons for design choices (colors, typography, chart types, interactivity) for Michael to incorporate in the slide deck. Visuals should have actionable titles and follow other class principles. 

**Michael:** Work on presentation flow and slide deck formatting. Also be the main person giving feedback on visuals made by Timmy and Andy. First half is a presentation to the intended audience. Second half is justifying design and storytelling choices. Exact time of presentation still TBD but can cut/add based on any changes.

**By Saturday evening**: Everyone review the dataset before the evening. If we like it, decide on the research question/problem statement/suggestion and intended audience. If not, pick new one and review together before deciding main idea and intended audience. Responsibilities can stay the same regardless of dataset choice. 

**By Tuesday evening**: Finish Visualizations and Write Ups

**By Wednesday evening**: Finish Slide Deck and Practice Presentation

# Timmy

# Andy